# Ajout des données Eco2mix à la BDD

In [2]:
import pandas as pd
import json
import os
from tqdm.notebook import tqdm
import numpy as np

Dates : 2015-01-01 -> 2025-03-06

In [3]:
data_dir = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline\process\data\ODRE\Eco2mix\FR'

In [ ]:
for date in tqdm(pd.date_range(start='2024-03-22', end='2024-03-22', freq='D', inclusive='both').date):
    for data_type in ['def', 'cons', 'tr']: 
        file = os.path.join(data_dir, f'{date.strftime(format='%Y%m%d')}_ODRE_Eco2mix-{data_type}_FR.json')
        if os.path.exists(file):
            break
    with open(file, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data['results'])
    df.date_heure = pd.to_datetime(df.date_heure)

    # if data['total_count'] != 96:
    #     print(f'File for {date.strftime(format='%Y%m%d')} has a total_count of {data["total_count"]}')
    # if len(data['results']) != 96:
    #     print(f'File for {date.strftime(format='%Y%m%d')} has {len(data["results"])} results')
    # start = pd.Timestamp(date).tz_localize('Europe/Paris').tz_convert('UTC')
    # end = pd.Timestamp(date + pd.Timedelta(days=1)).tz_localize('Europe/Paris').tz_convert('UTC')
    # time_range = pd.date_range(start=start, end=end, freq='15min', inclusive='left')
    # if not(np.array_equal(time_range.values, pd.to_datetime(df.date_heure).values)):
    #     print(date)

    # Remove duplicates
    df = df[['date_heure', 'consommation', 'eolien', 'solaire']].rename(columns={'date_heure' : 'start'}).groupby('date_heure').mean().tz_localize(None)
    # Fill in values for fields not at quarter-hour granularity (the 30min mean should stay the same)
    df.fillna(df.resample('30min').transform('mean'), inplace=True)
    # Add missing timestamps for the Summer to Winter time change
    df = df.resample('15min').asfreq()

    for col, col_name_in_BDD in zip(['eolien', 'solaire'], ['WIND', 'SOLAR']):
        part_df = df[[col]]
        # Fill missing values with linear interpolation and add a flag for missing values
        missing_mask = part_df.col.isna()
        part_df.loc[missing_mask, 'flag'] = 'Interpolated'
        part_df.loc[~missing_mask, 'flag'] = 'Unchanged'
        part_df.interpolate(method='time', inplace=True)

    # if df.resample('15min').asfreq().isna().all(axis=1).any():
    #     print(f'File for {date.strftime(format='%Y%m%d')} has missing data')



  0%|          | 0/1 [00:00<?, ?it/s]